Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

* The name of the team
* The total number of goals scored by the team during the 2011 season
* The total number of wins the team earned during the 2011 season
* A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)
* The team's win percentage on days where it was raining during games in the 2011 season.

In [1]:
from lib import MongoHandler, WeatherGetter
import pandas as pd
import sqlite3 
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


In [2]:
conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

Check the tables inside the database:

https://www.kaggle.com/laudanum/footballdelphi

In [3]:
print([i[0] for i in cur.execute("""SELECT * FROM sqlite_master;""").description])
cur.execute("""SELECT type, tbl_name FROM sqlite_master ORDER by type, tbl_name;""")
db = pd.DataFrame(cur.fetchall())
db.columns = [i[0] for i in cur.description]
db

['type', 'name', 'tbl_name', 'rootpage', 'sql']


,type,tbl_name
0,index,Matches
1,table,Matches
2,table,Teams
3,table,Teams_in_Matches
4,table,Unique_Teams
5,table,sqlite_sequence
6,view,FlatView
7,view,FlatView_Advanced
8,view,FlatView_Chrono_TeamOrder_Reduced


* ***Match_ID (int): unique ID per match***
* Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)
* ***Season (int): Season the match took place in (usually covering the period of August till May of the following year)***
* Date (str): Date of the match
* ***HomeTeam (str): Name of the home team***
* ***AwayTeam (str): Name of the away team***
* FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
* FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
* FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

In [4]:
cur.execute("""SELECT * FROM Matches WHERE Season = 2011;""")
df_ml = pd.DataFrame(cur.fetchall())
df_ml.columns = [i[0] for i in cur.description]
df_ml

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
...,...,...,...,...,...,...,...,...,...
987,44870,E0,2011,2012-05-13,Sunderland,Man United,0,1,A
988,44871,E0,2011,2012-05-13,Swansea,Liverpool,1,0,H
989,44872,E0,2011,2012-05-13,Tottenham,Fulham,2,0,H
990,44873,E0,2011,2012-05-13,West Brom,Arsenal,2,3,A


* ***Season (str): Football season for which the data is valid***   
* ***TeamName (str): Name of the team the data concerns***    
* KaderHome (str): Number of Players in the squad
* AvgAgeHome (str): Average age of players
* ForeignPlayersHome (str): Number of foreign players (non-German, non-English respectively) playing for the team
* OverallMarketValueHome (str): Overall market value of the team pre-season in EUR (based on data from transfermarkt.de)
* AvgMarketValueHome (str): Average market value (per player) of the team pre-season in EUR (based on data from transfermarkt.de)
* StadiumCapacity (str): Maximum stadium capacity of the team's home stadium

In [5]:
cur.execute("""SELECT * FROM Teams;""")
df_tl = pd.DataFrame(cur.fetchall())
df_tl.columns = [i[0] for i in cur.description]
df_tl.head()

,Season,TeamName,KaderHome,AvgAgeHome,ForeignPlayersHome,OverallMarketValueHome,AvgMarketValueHome,StadiumCapacity
0,2017,Bayern Munich,27,26,15,597950000,22150000,75000
1,2017,Dortmund,33,25,18,416730000,12630000,81359
2,2017,Leverkusen,31,24,15,222600000,7180000,30210
3,2017,RB Leipzig,30,23,15,180130000,6000000,42959
4,2017,Schalke 04,29,24,17,179550000,6190000,62271


* ***Match_ID (int): Unique match ID***  
* ***Unique_Team_ID (int): Unique team ID (This table is used to easily retrieve each match a given team has played in)***

In [6]:
cur.execute("""SELECT * FROM Teams_in_Matches;""")
df_tm = pd.DataFrame(cur.fetchall())
df_tm.columns = [i[0] for i in cur.description]
df_tm.head()

,Match_ID,Unique_Team_ID
0,1,26
1,1,46
2,2,26
3,2,42
4,3,26


* ***TeamName (str): Name of a team***  
* ***Unique_Team_ID (int): Unique identifier for each team***

In [7]:
cur.execute("""SELECT * FROM Unique_Teams;""")
df_ut = pd.DataFrame(cur.fetchall())
df_ut.columns = [i[0] for i in cur.description]
display(df_ut.info())
df_ut.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
TeamName          128 non-null object
Unique_Team_ID    128 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.1+ KB


None

,TeamName,Unique_Team_ID
0,Bayern Munich,1
1,Dortmund,2
2,Leverkusen,3
3,RB Leipzig,4
4,Schalke 04,5


In [8]:
#The name of the team

q = """SELECT Unique_Team_ID as Team_ID, TeamName FROM df_ut"""
df_1 = pysqldf(q)
df_1.head()

,Team_ID,TeamName
0,1,Bayern Munich
1,2,Dortmund
2,3,Leverkusen
3,4,RB Leipzig
4,5,Schalke 04


In [11]:
#The total number of goals scored by the team during the 2011 season


q1 = """SELECT HomeTeam as Team, sum(FTHG) as HG FROM df_ml GROUP by Team;"""

df_home = pysqldf(q1)
print(df_home.head())

q2 = """SELECT AwayTeam as Team, sum(FTAG) as AG FROM df_ml GROUP by Team;"""

df_away = pysqldf(q2)
print(df_away.head())


# q2 = """SELECT Team, h.HG, a.AG sum(FTHG+FTAG)
#         FROM df_home
#         JOIN
#         USING ()
#         GROUP by Team;"""

# df_goals = pysqldf(q2)
# print(df_goals.head())


            Team  HG
0         Aachen  15
1        Arsenal  39
2    Aston Villa  20
3       Augsburg  20
4  Bayern Munich  49
            Team  AG
0         Aachen  15
1        Arsenal  35
2    Aston Villa  17
3       Augsburg  16
4  Bayern Munich  28
